In [ ]:
import duckdb
import polars as pl  # Or you can use pandas here.

# NOTE: THE PYTHON API IS KIND OF "EH" SO WE'RE GOING TO USE THE SQL.

In [ ]:
# We can read json, csv, parquet, etc. into a Relation.
ddb_customers = duckdb.read_csv("./data/raw_customers.csv", header=True).set_alias(
    "customers"
)
ddb_orders = duckdb.read_csv("./data/raw_orders.csv", header=True).set_alias("orders")
ddb_items = duckdb.read_csv("./data/raw_items.csv", header=True).set_alias("items")
ddb_stores = duckdb.read_csv("./data/raw_stores.csv", header=True).set_alias("stores")
ddb_products = duckdb.read_csv("./data/raw_products.csv", header=True).set_alias(
    "products"
)

# # Basic Operations
ddb_customer_orders = duckdb.sql(
    """ 
SELECT                 
    customers.id
    , customers.name customer_name
    , products.type product_type
    , products.price
FROM
    ddb_customers customers 
    join ddb_orders orders ON customers.id = orders.customer 
    join ddb_items items ON orders.id = items.order_id
    join ddb_products products on items.sku = products.sku            
"""
)

In [ ]:
# Agg by name, product type.
ddb_agg_price = duckdb.sql(
    """
SELECT 
    customer_name
    , product_type 
    , sum(price) as total_price
FROM 
    ddb_customer_orders 
GROUP BY 
    customer_name
    , product_type
ORDER BY
    customer_name
    , product_type                       
"""
)

ddb_agg_price.show()

In [ ]:
# Persistent storage
# REF: https://duckdb.org/docs/api/python/dbapi
with duckdb.connect("example.db") as con:
    con.sql(
        "CREATE TABLE IF NOT EXISTS customers AS FROM read_csv('./data/raw_customers.csv', header=True, AUTO_DETECT=TRUE)"
    )

    results = con.query("SELECT * FROM customers LIMIT 5")
    print(results)